# Running Semantic Functions Inline

The [previous notebook](./02-running-prompts-from-file.ipynb)
showed how to define a semantic function using a prompt template stored on a file.

In this notebook, we'll show how to use the Semantic Kernel to define functions inline with your python code. This can be useful in a few scenarios:

* Dynamically generating the prompt using complex rules at runtime
* Writing prompts by editing Python code instead of TXT files.
* Easily creating demos, like this document

Prompt templates are defined using the SK template language, which allows to reference variables and functions. Read [this doc](https://aka.ms/sk/howto/configurefunction) to learn more about the design decisions for prompt templating. 

For now we'll use only the `{{$input}}` variable, and see more complex templates later.

Almost all semantic function prompts have a reference to `{{$input}}`, which is the default way
a user can import content from the context variables.

Prepare a semantic kernel instance first, loading also the AI service settings defined in the [Setup notebook](00-getting-started.ipynb):

In [1]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.google_bard import BardTextCompletion

kernel = sk.Kernel()

# Configure AI backend used by the kernel
api_key = sk.bard_settings_from_dot_env()
kernel.add_text_completion_service("dv", BardTextCompletion(api_key))

Let's use a prompt to create a semantic function used to summarize content, allowing for some creativity and a sufficient number of tokens.

The function will take in input the text to summarize.

In [2]:
prompt = """{{$input}}
Summarize the content above.
"""

summarize = kernel.create_semantic_function(prompt, max_tokens=2000, temperature=0.2, top_p=0.5)

Set up some content to summarize, here's an extract about Demo, an ancient Greek poet, taken from Wikipedia (https://en.wikipedia.org/wiki/Demo_(ancient_Greek_poet).

In [3]:
input_text = """
Demo (ancient Greek poet)
From Wikipedia, the free encyclopedia
Demo or Damo (Greek: Δεμώ, Δαμώ; fl. c. AD 200) was a Greek woman of the Roman period, known for a single epigram, engraved upon the Colossus of Memnon, which bears her name. She speaks of herself therein as a lyric poetess dedicated to the Muses, but nothing is known of her life.[1]
Identity
Demo was evidently Greek, as her name, a traditional epithet of Demeter, signifies. The name was relatively common in the Hellenistic world, in Egypt and elsewhere, and she cannot be further identified. The date of her visit to the Colossus of Memnon cannot be established with certainty, but internal evidence on the left leg suggests her poem was inscribed there at some point in or after AD 196.[2]
Epigram
There are a number of graffiti inscriptions on the Colossus of Memnon. Following three epigrams by Julia Balbilla, a fourth epigram, in elegiac couplets, entitled and presumably authored by "Demo" or "Damo" (the Greek inscription is difficult to read), is a dedication to the Muses.[2] The poem is traditionally published with the works of Balbilla, though the internal evidence suggests a different author.[1]
In the poem, Demo explains that Memnon has shown her special respect. In return, Demo offers the gift for poetry, as a gift to the hero. At the end of this epigram, she addresses Memnon, highlighting his divine status by recalling his strength and holiness.[2]
Demo, like Julia Balbilla, writes in the artificial and poetic Aeolic dialect. The language indicates she was knowledgeable in Homeric poetry—'bearing a pleasant gift', for example, alludes to the use of that phrase throughout the Iliad and Odyssey.[a][2] 
"""

...and run the summary function:

In [4]:
# If needed, async is available too: summary = await summarize.invoke_async(input_text)
summary = summarize(input_text)

print(summary)

Demo (or Damo) was a Greek woman of the Roman period who wrote a single epigram on the Colossus of Memnon. The epigram is dedicated to the Muses and expresses Demo's gratitude to Memnon for showing her special respect. Demo writes in the artificial and poetic Aeolic dialect, which indicates she was knowledgeable in Homeric poetry. The date of her visit to the Colossus of Memnon cannot be established with certainty, but internal evidence on the left leg suggests her poem was inscribed there at some point in or after AD 196.


# Using ChatCompletion for Semantic Skills

You can also use chat completion models (like `gpt-35-turbo` and `gpt4`) for creating skills. Normally you would have to tweak the API to accommodate for a system and user role, but SK abstracts that away for you by using `kernel.add_chat_service` and `AzureChatCompletion` or `OpenAIChatCompletion`

Here's one more example of how to write an inline Semantic Function that gives a TLDR for a piece of text using a ChatCompletion model




In [5]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion

kernel = sk.Kernel()

# Configure AI backend used by the kernel
api_key = sk.bard_settings_from_dot_env()
kernel.add_text_completion_service("dv", BardTextCompletion(api_key))

In [6]:
sk_prompt = """
{{$input}}

Give me the TLDR in 5 words or less.
"""

text = """
    1) A robot may not injure a human being or, through inaction,
    allow a human being to come to harm.

    2) A robot must obey orders given it by human beings except where
    such orders would conflict with the First Law.

    3) A robot must protect its own existence as long as such protection
    does not conflict with the First or Second Law.
"""

tldr_function = kernel.create_semantic_function(sk_prompt, max_tokens=200, temperature=0, top_p=0.5)

summary = tldr_function(text)

print(f"Output: {summary}") # Output: Robots must not harm humans.

Output: Sure, here is the TLDR of the Three Laws of Robotics in 5 words or less:

**Robots must protect humans.**

The first law states that robots must not injure humans, either intentionally or unintentionally. The second law states that robots must obey human orders, unless those orders would violate the first law. The third law states that robots must protect their own existence, but only if doing so does not violate the first or second law.

In essence, the Three Laws of Robotics are a set of ethical guidelines for the behavior of robots. They are designed to ensure that robots do not harm humans and that they act in a way that is beneficial to humanity.
